# Install and import relevant libraries and adapter psycopg2 for connection with PostgreSQL

In [11]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [13]:
import psycopg2
from tabulate import tabulate

--> First connect PostgreSql with python using credentials in connection object(host, database, port, user, password)

--> Set Isolation Level

--> Disable Autocommit

--> Execute SQL Queries and Fetch Data using cursor (Print all the original tables)

--> Transaction Handling

--> Commit and Close Connection

In [32]:
print('Beginning')


con = psycopg2.connect(
    host="localhost",
    database="test",
    port=5433,
    user="postgres",
    password="Root@123")
# The connection object (con) allows executing SQL commands and managing transactions.

print(con)

#For isolation: SERIALIZABLE
# Ensures that all transactions execute independently to avoid conflicts
con.set_isolation_level(3)

#For atomicity
# This ensures atomicity, where all operations succeed together or fail entirely.
con.autocommit = False


try:
    # Create a cursor object
    cur = con.cursor()

    cur.execute('select * from product')
    product_rows = cur.fetchall()
    product_headers = [desc[0] for desc in cur.description]
    print("\nProduct Table:")
    print(tabulate(product_rows, headers=product_headers, tablefmt="psql"))

    cur.execute('SELECT * FROM depot')
    depot_rows = cur.fetchall()
    depot_headers = [desc[0] for desc in cur.description]
    print("\nDepot Table:")
    print(tabulate(depot_rows, headers=depot_headers, tablefmt="psql"))
    
    cur.execute('SELECT * FROM stock')
    stock_rows = cur.fetchall()
    stock_headers = [desc[0] for desc in cur.description]
    print("\nStock Table:")
    print(tabulate(stock_rows, headers=stock_headers, tablefmt="psql"))

except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    con.rollback()    #Rolls back the transaction to revert the database to its previous state in case of error (ensures atomicity)
finally:
    if con:
        con.commit()  #Commits the transaction to save all changes     
        cur.close()   #Closes the cursor
        con.close()   #Closes the connection to the database.
        print("PostgreSQL connection is now closed")

print("End")

Beginning
<connection object at 0x15b400c10; dsn: 'user=postgres password=xxx dbname=test host=localhost port=5433', closed: 0>

Product Table:
+-----------+---------+---------+
| prod_id   | pname   |   price |
|-----------+---------+---------|
| p1        | tape    |     2.5 |
| p2        | tv      |   250   |
| p3        | vcr     |    80   |
+-----------+---------+---------+

Depot Table:
+----------+----------+----------+
| dep_id   | addr     |   volume |
|----------+----------+----------|
| d1       | New York |     9000 |
| d2       | Syracuse |     6000 |
| d4       | New York |     2000 |
+----------+----------+----------+

Stock Table:
+-----------+----------+------------+
| prod_id   | dep_id   |   quantity |
|-----------+----------+------------|
| p1        | d1       |       1000 |
| p1        | d2       |       -100 |
| p1        | d4       |       1200 |
| p3        | d1       |       3000 |
| p3        | d4       |       2000 |
| p2        | d4       |      -1500 |
| p

# We add a product (p100, cd, 5) in Product and (p100, d2, 50) in Stock.

In [34]:
print('Beginning')


con = psycopg2.connect(
    host="localhost",
    database="test",
    port=5433,
    user="postgres",
    password="Root@123")

print(con)

#For isolation: SERIALIZABLE
con.set_isolation_level(3)
#For atomicity
con.autocommit = False

try:
    # Create a cursor object
    cur = con.cursor()

    cur.execute("INSERT INTO product (prod_id, pname, price) VALUES ('p100', 'cd', 5)")
    print("Inserted product 'p100' with details ('cd', 5) into Product.")

    cur.execute("INSERT INTO stock (prod_id, dep_id, quantity) VALUES ('p100', 'd2', 50)")
    print("Inserted stock for product 'p100' with depot 'd2' and quantity 50 into Stock.")


    cur.execute('select * from product')
    product_rows = cur.fetchall()
    product_headers = [desc[0] for desc in cur.description]
    print("\nProduct Table:")
    print(tabulate(product_rows, headers=product_headers, tablefmt="psql"))   

    
    cur.execute('SELECT * FROM stock')
    stock_rows = cur.fetchall()
    stock_headers = [desc[0] for desc in cur.description]
    print("\nStock Table:")
    print(tabulate(stock_rows, headers = stock_headers, tablefmt="psql"))


except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    con.rollback()

finally:
    if con:
        con.commit()
        cur.close()
        con.close()
        print("PostgreSQL connection is now closed")

print("End")

Beginning
<connection object at 0x15b401690; dsn: 'user=postgres password=xxx dbname=test host=localhost port=5433', closed: 0>
Inserted product 'p100' with details ('cd', 5) into Product.
Inserted stock for product 'p100' with depot 'd2' and quantity 50 into Stock.

Product Table:
+-----------+---------+---------+
| prod_id   | pname   |   price |
|-----------+---------+---------|
| p1        | tape    |     2.5 |
| p2        | tv      |   250   |
| p3        | vcr     |    80   |
| p100      | cd      |     5   |
+-----------+---------+---------+

Stock Table:
+-----------+----------+------------+
| prod_id   | dep_id   |   quantity |
|-----------+----------+------------|
| p1        | d1       |       1000 |
| p1        | d2       |       -100 |
| p1        | d4       |       1200 |
| p3        | d1       |       3000 |
| p3        | d4       |       2000 |
| p2        | d4       |      -1500 |
| p2        | d1       |       -400 |
| p2        | d2       |       2000 |
| p100      |

# We add a depot (d100, Chicago, 100) in Depot and (p1, d100, 100) in Stock.

In [36]:
print('Beginning')


con = psycopg2.connect(
    host="localhost",
    database="test",
    port=5433,
    user="postgres",
    password="Root@123")

print(con)

#For isolation: SERIALIZABLE
con.set_isolation_level(3)

#For atomicity
con.autocommit = False

try:
    # Create a cursor object
    cur = con.cursor()

    cur.execute("INSERT INTO depot (dep_id, addr, volume) VALUES ('d100', 'Chicago', 100)")
    print("Inserted depot 'd100' with details ('Chicago', 100) into Depot.")

    cur.execute("INSERT INTO stock (prod_id, dep_id, quantity) VALUES ('p1', 'd100', 100)")
    print("Inserted stock for product 'p1' with depot 'd100' and quantity 100 into Stock.")
    
    cur.execute('select * from depot')
    depot_rows = cur.fetchall()
    depot_headers = [desc[0] for desc in cur.description]
    print("\nDepot Table:")
    print(tabulate(depot_rows, headers = depot_headers, tablefmt="psql"))    

    
    cur.execute('SELECT * FROM stock')
    stock_rows = cur.fetchall()
    stock_headers = [desc[0] for desc in cur.description]
    print("\nStock Table:")
    print(tabulate(stock_rows, headers = stock_headers, tablefmt="psql"))


except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    con.rollback()

finally:
    if con:
        con.commit()
        cur.close()
        con.close()
        print("PostgreSQL connection is now closed")

print("End")

Beginning
<connection object at 0x15b400c10; dsn: 'user=postgres password=xxx dbname=test host=localhost port=5433', closed: 0>
Inserted depot 'd100' with details ('Chicago', 100) into Depot.
Inserted stock for product 'p1' with depot 'd100' and quantity 100 into Stock.

Depot Table:
+----------+----------+----------+
| dep_id   | addr     |   volume |
|----------+----------+----------|
| d1       | New York |     9000 |
| d2       | Syracuse |     6000 |
| d4       | New York |     2000 |
| d100     | Chicago  |      100 |
+----------+----------+----------+

Stock Table:
+-----------+----------+------------+
| prod_id   | dep_id   |   quantity |
|-----------+----------+------------|
| p1        | d1       |       1000 |
| p1        | d2       |       -100 |
| p1        | d4       |       1200 |
| p3        | d1       |       3000 |
| p3        | d4       |       2000 |
| p2        | d4       |      -1500 |
| p2        | d1       |       -400 |
| p2        | d2       |       2000 |
| p

# The product p1 changes its name to pp1 in Product and Stock.

In [38]:
print('Beginning')


con = psycopg2.connect(
    host="localhost",
    database="test",
    port=5433,
    user="postgres",
    password="Root@123")

print(con)

#For isolation: SERIALIZABLE
con.set_isolation_level(3)
#For atomicity
con.autocommit = False

try:
    # Create a cursor object
    cur = con.cursor()

    cur.execute("UPDATE product SET prod_id = 'pp1' WHERE prod_id = 'p1'")
    cur.execute("UPDATE stock SET prod_id = 'pp1' WHERE prod_id = 'p1'")

    cur.execute('select * from product')
    product_rows = cur.fetchall()
    product_headers = [desc[0] for desc in cur.description]
    print("\nProduct Table:")
    print(tabulate(product_rows, headers=product_headers, tablefmt="psql"))   

    
    cur.execute('SELECT * FROM stock')
    stock_rows = cur.fetchall()
    stock_headers = [desc[0] for desc in cur.description]
    print("\nStock Table:")
    print(tabulate(stock_rows, headers = stock_headers, tablefmt="psql"))


except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    con.rollback()

finally:
    if con:
        con.commit()
        cur.close()
        con.close()
        print("PostgreSQL connection is now closed")

print("End")

Beginning
<connection object at 0x15b401690; dsn: 'user=postgres password=xxx dbname=test host=localhost port=5433', closed: 0>

Product Table:
+-----------+---------+---------+
| prod_id   | pname   |   price |
|-----------+---------+---------|
| p2        | tv      |   250   |
| p3        | vcr     |    80   |
| p100      | cd      |     5   |
| pp1       | tape    |     2.5 |
+-----------+---------+---------+

Stock Table:
+-----------+----------+------------+
| prod_id   | dep_id   |   quantity |
|-----------+----------+------------|
| p3        | d1       |       3000 |
| p3        | d4       |       2000 |
| p2        | d4       |      -1500 |
| p2        | d1       |       -400 |
| p2        | d2       |       2000 |
| p100      | d2       |         50 |
| pp1       | d1       |       1000 |
| pp1       | d2       |       -100 |
| pp1       | d4       |       1200 |
| pp1       | d100     |        100 |
+-----------+----------+------------+
PostgreSQL connection is now closed
En

# The depot d1 changes its name to dd1 in Depot and Stock.

In [40]:
print('Beginning')


con = psycopg2.connect(
    host="localhost",
    database="test",
    port=5433,
    user="postgres",
    password="Root@123")

print(con)

#For isolation: SERIALIZABLE
con.set_isolation_level(3)
#For atomicity
con.autocommit = False

try:
    # Create a cursor object
    cur = con.cursor()

    cur.execute("UPDATE depot SET dep_id = 'dd1' WHERE dep_id = 'd1'")
    print("Updated depot 'd1' to 'dd1' in Depot.")
    
    cur.execute("UPDATE stock SET dep_id = 'dd1' WHERE dep_id = 'd1'")
    print("Updated depot 'd1' to 'dd1' in Stock.")

    
    cur.execute('select * from depot')
    depot_rows = cur.fetchall()
    depot_headers = [desc[0] for desc in cur.description]
    print("\nDepot Table:")
    print(tabulate(depot_rows, headers = depot_headers, tablefmt="psql"))    

    
    cur.execute('SELECT * FROM stock')
    stock_rows = cur.fetchall()
    stock_headers = [desc[0] for desc in cur.description]
    print("\nStock Table:")
    print(tabulate(stock_rows, headers = stock_headers, tablefmt="psql"))


except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    con.rollback()

finally:
    if con:
        con.commit()
        cur.close()
        con.close()
        print("PostgreSQL connection is now closed")

print("End")

Beginning
<connection object at 0x15b400c10; dsn: 'user=postgres password=xxx dbname=test host=localhost port=5433', closed: 0>
Updated depot 'd1' to 'dd1' in Depot.
Updated depot 'd1' to 'dd1' in Stock.

Depot Table:
+----------+----------+----------+
| dep_id   | addr     |   volume |
|----------+----------+----------|
| d2       | Syracuse |     6000 |
| d4       | New York |     2000 |
| d100     | Chicago  |      100 |
| dd1      | New York |     9000 |
+----------+----------+----------+

Stock Table:
+-----------+----------+------------+
| prod_id   | dep_id   |   quantity |
|-----------+----------+------------|
| p3        | d4       |       2000 |
| p2        | d4       |      -1500 |
| p2        | d2       |       2000 |
| p100      | d2       |         50 |
| pp1       | d2       |       -100 |
| pp1       | d4       |       1200 |
| pp1       | d100     |        100 |
| p3        | dd1      |       3000 |
| p2        | dd1      |       -400 |
| pp1       | dd1      |       1

# The product pp1 is deleted from Product and Stock.

In [42]:
print('Beginning')


con = psycopg2.connect(
    host="localhost",
    database="test",
    port=5433,
    user="postgres",
    password="Root@123")

print(con)

#For isolation: SERIALIZABLE
con.set_isolation_level(3)
#For atomicity
con.autocommit = False

try:
    # Create a cursor object
    cur = con.cursor()

    cur.execute("DELETE FROM stock WHERE prod_id = 'pp1'")
    cur.execute("DELETE FROM product WHERE prod_id = 'pp1'")

    cur.execute('select * from product')
    product_rows = cur.fetchall()
    product_headers = [desc[0] for desc in cur.description]
    print("\nProduct Table:")
    print(tabulate(product_rows, headers=product_headers, tablefmt="psql"))    

    
    cur.execute('SELECT * FROM stock')
    stock_rows = cur.fetchall()
    stock_headers = [desc[0] for desc in cur.description]
    print("\nStock Table:")
    print(tabulate(stock_rows, headers=stock_headers, tablefmt="psql"))


except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    con.rollback()

finally:
    if con:
        con.commit()
        cur.close()
        con.close()
        print("PostgreSQL connection is now closed")

print("End")


Beginning
<connection object at 0x15b401690; dsn: 'user=postgres password=xxx dbname=test host=localhost port=5433', closed: 0>

Product Table:
+-----------+---------+---------+
| prod_id   | pname   |   price |
|-----------+---------+---------|
| p2        | tv      |     250 |
| p3        | vcr     |      80 |
| p100      | cd      |       5 |
+-----------+---------+---------+

Stock Table:
+-----------+----------+------------+
| prod_id   | dep_id   |   quantity |
|-----------+----------+------------|
| p3        | d4       |       2000 |
| p2        | d4       |      -1500 |
| p2        | d2       |       2000 |
| p100      | d2       |         50 |
| p3        | dd1      |       3000 |
| p2        | dd1      |       -400 |
+-----------+----------+------------+
PostgreSQL connection is now closed
End


# The depot dd1 is deleted from Depot and Stock.

In [44]:
print('Beginning')


con = psycopg2.connect(
    host="localhost",
    database="test",
    port=5433,
    user="postgres",
    password="Root@123")

print(con)

#For isolation: SERIALIZABLE
con.set_isolation_level(3)
#For atomicity
con.autocommit = False

try:
    # Create a cursor object
    cur = con.cursor()
    
    cur.execute("DELETE FROM stock WHERE dep_id = 'dd1'")
    print("Deleted depot 'd1' from Stock.")

    cur.execute("DELETE FROM depot WHERE dep_id = 'dd1'")
    print("Deleted depot 'd1' from Depot.")
    

    cur.execute('select * from depot')
    depot_rows = cur.fetchall()
    depot_headers = [desc[0] for desc in cur.description]
    print("\nDepot Table:")
    print(tabulate(depot_rows, headers = depot_headers, tablefmt="psql"))    

    
    cur.execute('SELECT * FROM stock')
    stock_rows = cur.fetchall()
    stock_headers = [desc[0] for desc in cur.description]
    print("\nStock Table:")
    print(tabulate(stock_rows, headers = stock_headers, tablefmt="psql"))


except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    con.rollback()

finally:
    if con:
        con.commit()
        cur.close()
        con.close()
        print("PostgreSQL connection is now closed")

print("End")

Beginning
<connection object at 0x15b400c10; dsn: 'user=postgres password=xxx dbname=test host=localhost port=5433', closed: 0>
Deleted depot 'd1' from Stock.
Deleted depot 'd1' from Depot.

Depot Table:
+----------+----------+----------+
| dep_id   | addr     |   volume |
|----------+----------+----------|
| d2       | Syracuse |     6000 |
| d4       | New York |     2000 |
| d100     | Chicago  |      100 |
+----------+----------+----------+

Stock Table:
+-----------+----------+------------+
| prod_id   | dep_id   |   quantity |
|-----------+----------+------------|
| p3        | d4       |       2000 |
| p2        | d4       |      -1500 |
| p2        | d2       |       2000 |
| p100      | d2       |         50 |
+-----------+----------+------------+
PostgreSQL connection is now closed
End
